In [1]:
# Para usar o sleep:
import time
# Para logs:
from tqdm import tqdm
import logging
import sys
sys.path.append('../utils')
from utils import get_logger, Clock, load_json, export_json
# Para web scraping:
import requests
from bs4 import BeautifulSoup, SoupStrainer
import cchardet
# Para acessar as credenciais de login de um arquivo externo:
import os
import dotenv
# Para navegar na página além do botão 'Show more':
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
# Contador
from itertools import count
# Manipulação de JSON:
import json
from dictdiffer import diff
# Para calcular similaridade entre strings:
from thefuzz import fuzz

In [2]:
logger = logging.getLogger(__name__)
logger = get_logger(logger=logger)

In [3]:
base_url = 'https://untappd.com'
headers = {
    "User-Agent":
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36 OPR/97.0.0.0"
}

In [4]:
path = '../data/'
file_number = "_1"
file_name = "users_second_level" + file_number + ".json"

data = load_json(path + file_name)

In [5]:
all_users = data['users']


url = "https://servicodados.ibge.gov.br/api/v1/localidades/municipios"
response = requests.get(url)
brazilian_cities = []

if response.status_code == 200:
    data = response.json()
    for city in data:
        brazilian_cities.append(city["nome"])
else:
    print("Falha na solicitação à API")

In [6]:
dotenv.load_dotenv()
EMAIL = os.getenv('EMAIL')
PASSWORD = os.getenv('PASSWORD')

options = Options()
options.add_argument("--disable-infobars")
options.add_argument("start-maximized")
options.add_argument("--disable-extensions")
options.add_experimental_option("prefs", {"profile.default_content_setting_values.notifications": 1})

In [8]:
#browser = webdriver.Chrome(options=options)

#browser.get(base_url + '/login')
#browser.find_element(By.NAME,"username").send_keys(EMAIL)
#browser.find_element(By.NAME,"password").send_keys(PASSWORD)
#time.sleep(30)

Filtrando os IDs de usuários que são de cidades brasileiras ou têm grandes chances de ser (similaridade de 80% pelo menos entre os nomes das localizações e alguma cidade do Brasil) e salvando numa lista filtrada, bem como um dicionário para revisão:

In [ ]:
brazilian_users = []
to_review = {}

logger.info("Iniciando uma sessão...")
with requests.Session() as session:
    logger.info("A iteração começará agora...")
    iteration_time = Clock("Filtragem dos IDs de usuários")
    with tqdm(total=len(all_users)) as progress_bar:
        progress_bar.set_description("Filtrando apenas os IDs de usuários que são de cidades brasileiras")
        for user in all_users:
            try:
                page = requests.get(base_url + user, headers=headers)
                soup = BeautifulSoup(page.text, 'html.parser')
                location = soup.select('div.user-details > p.location')[0].get_text()
                if location != '':
                    if location in brazilian_cities:
                        logger.info(f"Usuário brasileiro encontrado ({user}, cidade: {location})! Adicionando à nova lista...")
                        brazilian_users.append(user)
                    else:
                        similarity_threshold = 80
                        for brazilian_city in brazilian_cities:
                            similarity = fuzz.ratio(location, brazilian_city)
                            if similarity >= similarity_threshold:
                                logger.info(f"Possível usuário brasileiro encontrado ({user}, cidade: {location}, {similarity}% de similaridade com {brazilian_city})! Adicionando à nova lista...")
                                brazilian_users.append(user)
                                logger.info("Adicionando também ao dicionário de revisão manual...")
                                to_review[user]= location
                                file_to_review = "users_second_level_to_review" + file_number + ".json"
                                export_json(to_review, path + file_to_review)
                                break
                progress_bar.update(1)
                possible_br_users_json = {'possible_br_users_second_level_': brazilian_users}
                possible_br_users_file_name = "possible_brazilian_second_level_beer_user_links" + file_number + ".json"
                export_json(possible_br_users_json, path + possible_br_users_file_name)
            except IndexError:
                print(f"Elemento não encontrado para o usuário: {user}")
                progress_bar.update(1)
                continue
    iteration_time.stop_watch()
logger.info(f"Operação concluída! A lista filtrada possui {len(brazilian_users)} IDs de usuários.")

In [ ]:
logger.info(f"Dos {len(brazilian_users)} usuários resultantes, {len(to_review)} ({len(to_review)/len(brazilian_users)*100:.2f}%) foram adicionados com base na similaridade e devem ser revisados.")

Exportando os dados:

In [ ]:
possible_br_users_json = {'possible_br_users_second_level_': brazilian_users}
possible_br_users_file_name = "possible_brazilian_second_level_beer_user_links" + file_number + ".json"

logger.info('Salvando a lista de IDs de usuários filtrados num arquivo JSON...')
export_json(possible_br_users_json, path + possible_br_users_file_name)
logger.info("Exportação concluída! O arquivo resultante se encontra dentro da pasta 'data' com o nome 'possible_brazilian_second_level_beer_user_links.json'.")


file_to_review = "users_second_level_to_review" + file_number + ".json"
logger.info('Salvando a lista de IDs de usuários que precisam de revisão num arquivo JSON...')
export_json(to_review, path + file_to_review)
logger.info("Exportação concluída! O arquivo resultante se encontra dentro da pasta 'data' com o nome 'users_second_level_to_review.json'.")

### Removendo usuários cuja similaridade da cidade era elevada mas não são brasileiros:

Carregando o dicionário de revisão original e uma versão manualmente filtrada:

In [ ]:
#path = '../data/'
#file_to_review = "users_second_level_to_review.json"
#file_to_review_filtered = "users_second_level_to_review_filtered.json"

#to_review = load_json(path + file_to_review)
#to_review_filtered = load_json(path + file_to_review_filtered)

Agora, vamos filtrar a lista de usuários completa para remover os que não são brasileiros e exportá-la:

In [ ]:
#logger.info('Extraindo os usuários que não são brasileiros...')
#non_brazilian_users = {user: location for user, location in list(diff(to_review, to_review_filtered))[0][2]}
#logger.info(f'{len(non_brazilian_users)} usuários não são brasileiros!')

In [ ]:
#brazilian_users = ([user for user in brazilian_users if user not in set(non_brazilian_users.keys())])
#logger.info(f'Após a filtragem, a lista de usuários brasileiros passou a ter {len(brazilian_users)} IDs de usuários')

In [ ]:
#logger.info('Exportando a lista de usuários brasileiros...')
#brazilian_users_json = {'brazilian_users': brazilian_users}
#brazilian_users_file_name = 'brazilian_users_second_level.json'
#export_json(brazilian_users_json, path + brazilian_users_file_name)
#logger.info('Exportação concluída!')